In [ ]:
import pandas as pd
import numpy as np
import json
from urllib.request import urlopen
import datetime #to change date format
import re #regex
import sqlalchemy
import sqlite3

In [ ]:
# store the API url 

url = 'https://www.gov.uk/bank-holidays.json'

In [ ]:
# store the response of URL

response = urlopen(url)

In [ ]:
# storing the JSON response 

data_json = json.loads(response.read())

In [ ]:
# print the json response

data_json

In [ ]:
# use .json_normalize to attempt to shape the json response into a flat table

data_norm = pd.json_normalize(data_json)

In [ ]:
# view the attempted normalised table

data_norm

In [ ]:
# get a list of the column names

column_names = list(data_norm.columns)

In [ ]:
# view the list of column names

column_names

In [ ]:
# create a list of divisions

divisions_list = [column_names[1], column_names[3], column_names[5]]

In [ ]:
# view the divisions list

divisions_list

In [ ]:
# make a global variable to be used in connecting to the SQL database

global engine 
engine = sqlalchemy.create_engine('sqlite:///BankHolidaysAPI_v2.db')

In [ ]:
# connect to the database

conn = sqlite3.connect('BankHolidaysAPI_v2.db')

In [ ]:
# function to cleanse the data:
    # save the function input as division_name
    # subset the normalised dataframe depending on the division name and turn it into a list datatype
    # transpose the subsetted dataframe
    # change the only column in the transposed dataframe to string
    # remove {} brackets at start and end of each row
    # text-to-column split by comma
    # remove the unnecessary leading and trailing characters for each row
    # rename each column with more appropriate names
    # add a column called Division based on the division name used as funcion input
    # save each cleansed dataframe to a SQL database
    # remove unwanted trailing characters from Division column

def cleanse(x):
    division_name = x
    data_x = pd.DataFrame(data_norm[x].values.tolist(), index = data_norm.index)
    data_x_T = data_x.T
    data_x_T[0] = data_x_T[0].astype(str)
    data_x_T[0] = data_x_T[0].str.replace('[{}]', '')
    x = data_x_T[0].str.split(',', expand = True)
    x[0] = x[0].str[10:]
    x[0] = x[0].str[:-1]
    x[1] = x[1].str[10:]
    x[1] = x[1].str[:-1]
    x[2] = x[2].str[11:]
    x[2] = x[2].str[:-1]
    x[3] = x[3].str[11:]
    df_x = x.rename(columns = {0:'Title', 1:'Date', 2:'Notes', 3:'Bunting'})
    df_x['Division'] = division_name[:-7]
    df_x['Date'] = pd.to_datetime(df_x['Date']).dt.strftime('%d-%m-%Y')
    return df_x.to_sql(division_name, engine, if_exists = 'replace', index = False)

In [ ]:
# use list of divisions to parse through the cleanse() function i.e. perform cleanse() function for each division

for d in divisions_list:
    cleanse(d)

In [ ]:
# view the tables in the SQL database

con = sqlite3.connect('BankHolidaysAPI_v2.db')
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

In [ ]:
# example table

ew = pd.read_sql('england-and-wales.events', engine)
s = pd.read_sql('scotland.events', engine)
ni = pd.read_sql('northern-ireland.events', engine)

ew

In [ ]:
len(ew)+len(s)+len(ni)

In [ ]:
Data_SQL = pd.concat([ew, s, ni])